In [1]:
import tensorflow as tf
import scipy, numpy

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

2023-11-24 19:46:36.301128: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-24 19:46:40.829405: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-24 19:46:40.841921: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-24 19:46:51.318049: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Configurações para o treinamento
train_data_dir = './data'
batch_size = 32 # 32 imagens por lote
num_epochs = 100 # 100 épocas de treinamento
image_size = (150, 150) # 150x150 pixels
num_classes = 2 # 2 classes: 'Papai Noel' e 'Não Papai Noel'

In [3]:
train_datagen = ImageDataGenerator(
    rescale = 1.0/255,
    validation_split = 0.25, # 25% para validação
    shear_range = 0.2, # inclinação
    zoom_range = 0.2, # zoom
    horizontal_flip = True # espelhamento horizontal
)

In [4]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'binary',
    subset = 'training'
)

Found 462 images belonging to 2 classes.


In [5]:
validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'binary',
    subset = 'validation'
)

Found 152 images belonging to 2 classes.


In [6]:
model = Sequential()

model.add(Conv2D(64, (3, 3), input_shape=(image_size[0], image_size[1], 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

In [7]:
# compilar o modelo
model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy', # mais próximo de 0 melhor, 0.002 por exemplo
    metrics = ['accuracy']
)

In [8]:
# treinamento
model.fit(
    train_generator,
    epochs = num_epochs,
    validation_data = validation_generator
)

print('Treinamento concluído!')

Epoch 1/100
15/15 [==============================] - 31s 2s/step - loss: 0.6862 - accuracy: 0.5498 - val_loss: 0.6142 - val_accuracy: 0.8487
Epoch 2/100
15/15 [==============================] - 32s 2s/step - loss: 0.6393 - accuracy: 0.7359 - val_loss: 0.5227 - val_accuracy: 0.8224
Epoch 3/100
15/15 [==============================] - 30s 2s/step - loss: 0.5715 - accuracy: 0.8528 - val_loss: 0.4729 - val_accuracy: 0.8421
Epoch 4/100
15/15 [==============================] - 31s 2s/step - loss: 0.5337 - accuracy: 0.8377 - val_loss: 0.4134 - val_accuracy: 0.8816
Epoch 5/100
15/15 [==============================] - 30s 2s/step - loss: 0.4493 - accuracy: 0.8528 - val_loss: 0.2781 - val_accuracy: 0.9013
Epoch 6/100
15/15 [==============================] - 26s 2s/step - loss: 0.3751 - accuracy: 0.8853 - val_loss: 0.3031 - val_accuracy: 0.8421
Epoch 7/100
15/15 [==============================] - 26s 2s/step - loss: 0.4984 - accuracy: 0.8203 - val_loss: 0.3889 - val_accuracy: 0.8487
Epoch 8/100
1

In [9]:
# salvar o modelo
model.save('./models/cfIsNoel-RCastilhos.keras')